In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy.engine import create_engine


In [ ]:
dfEJ = pd.read_xml(path_or_buffer="../../../data_test/finess/simple/finess_cs1400101_stock_20230126-0330.xml", xpath="./structureej")

In [ ]:
# Ne garder que les EJ ouvert
dfEJ = dfEJ[dfEJ["datefermeture"].isna()]


In [ ]:
# Affiche le nombre d'EJ par statut
dfEJ.groupby("statutjuridique")["libstatutjuridique"].agg(["count", "min"]).rename(columns={"min": "libstatutjuridique"})


In [ ]:
# Affiche les codes des status juridiques
dfStatutJuridique = pd.read_xml(
    path_or_buffer="../../../data_test/finess/nomenclature/finess_cs1500107_stock_20230126-0334.xml", xpath="./nomenclstatutavecagr"
)
dfStatutJuridique = dfStatutJuridique[["code", "codeagr3", "codeagr2", "codeagr1"]].rename(columns={"code": "statutjuridique"})
dfStatutJuridique.head()


In [ ]:
# Joint les entités juridiques avec la nomenclature des statuts juridiques pour avoir leur niveau 1, 2 et 3.
dfMerged = dfEJ.merge(dfStatutJuridique, on="statutjuridique")
dfMerged.head()


In [ ]:
# Affiche uniquement les columns liées aux statuts juridiques
dfEJStatut = dfMerged[["nofiness", "libstatutjuridique", "statutjuridique", "codeagr1", "codeagr2", "codeagr3"]]
dfEJStatut.sample(5)

In [ ]:
# Vérifie que tous les EJ sont associés à un code niv 1 2 et 3

# 1 - On s'assure qu'il y a autant d'EJ dans celui d'origine que dans le dataframe mergé
display(dfEJStatut.shape)
display(dfEJ.shape)


In [ ]:
# 2 - On s'assure que chaque EJ a bien un niv 1, 2 et 3
display(dfEJStatut.groupby(["codeagr1"])["codeagr1"].count())
display(dfEJStatut.groupby(["codeagr2"])["codeagr2"].count())
display(dfEJStatut.groupby(["codeagr3"])["codeagr3"].count())


In [ ]:
# Association des tags publics, privée non lucratif, privé lucratif
dfWithTag = dfEJStatut.copy()
conditions = [(dfWithTag["codeagr1"] == 1000), (dfWithTag["codeagr2"] == 2100), (dfWithTag["codeagr2"] == 2200), (dfWithTag["codeagr1"] == 3000)]
tagValues = ["Public", "Privé non Lucratif", "Privé Lucratif", "Personne morale de droit étranger"]
dfWithTag["Tag"] = np.select(conditions, tagValues)
dfWithTag.sample(10)


In [ ]:
# Vérification que chaque EJ est bien associé à un tag
nbEJParTag = dfWithTag.groupby("Tag")[["nofiness"]].count()
display(nbEJParTag)
display("total :" + str(dfWithTag.groupby("Tag").count().nofiness.sum()) + " / " + str(dfEJ.shape[0]))


In [ ]:
# Affichage du nombre d'EJ par tag
nbEJParTag.plot(kind="bar")


In [ ]:
# Les EJ personne morale de droit étranger encore ouvert
noFinessDE = dfWithTag.loc[dfWithTag["Tag"] == "Personne morale de droit étranger"]["nofiness"]
dfEJ[dfEJ.nofiness.isin(noFinessDE)][["nofiness", "datefermeture"]]
